In [17]:
# Data preprocessing libraries
import numpy as np
import pandas as pd
from pandas.plotting import parallel_coordinates

import os
import sqlite3
import math
from collections import Counter
from pathlib import Path
from tqdm import tqdm

# Model
from scipy.stats import skew
import yellowbrick
import sklearn
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV




In [18]:
# Reading csv files and drop the first column
df_train = pd.read_csv(r"fraud test.csv")
df_train.drop(df_train.columns[0], axis=1, inplace=True)

#df_test = pd.read_csv(r"fraudTest.csv")
#df_test.drop(df_test.columns[0], axis=1, inplace=True)

# First view 10 rows
df_train.head(10)

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,21/06/2020 12:14,2.291160e+15,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,...,33.9659,-80.9355,333497,Mechanical engineer,19/03/1968,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,21/06/2020 12:14,3.573030e+15,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,...,40.3207,-110.4360,302,"Sales professional, IT",17/01/1990,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,21/06/2020 12:14,3.598220e+15,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,...,40.6729,-73.5365,34496,"Librarian, public",21/10/1970,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,21/06/2020 12:15,3.591920e+15,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,Titusville,...,28.5697,-80.8191,54767,Set designer,25/07/1987,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,21/06/2020 12:15,3.526830e+15,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,Falmouth,...,44.2529,-85.0170,1126,Furniture designer,06/07/1955,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0
5,21/06/2020 12:15,3.040770e+13,fraud_Daugherty LLC,kids_pets,19.55,Danielle,Evans,F,76752 David Lodge Apt. 064,Breesport,...,42.1939,-76.7361,520,Psychotherapist,13/10/1991,798db04aaceb4febd084f1a7c404da93,1371816937,41.747157,-77.584197,0
6,21/06/2020 12:15,2.131810e+14,fraud_Romaguera Ltd,health_fitness,133.93,Kayla,Sutton,F,010 Weaver Land,Carlotta,...,40.5070,-123.9743,1139,"Therapist, occupational",15/01/1951,17003d7ce534440eadb10c4750e020e5,1371816944,41.499458,-124.888729,0
7,21/06/2020 12:15,3.589290e+15,fraud_Reichel LLC,personal_care,10.37,Paula,Estrada,F,350 Stacy Glens,Spencer,...,43.7557,-97.5936,343,"Development worker, international aid",05/03/1972,8be473af4f05fc6146ea55ace73e7ca2,1371816950,44.495498,-97.728453,0
8,21/06/2020 12:16,3.596360e+15,"fraud_Goyette, Howell and Collier",shopping_pos,4.37,David,Everett,M,4138 David Fall,Morrisdale,...,41.0001,-78.2357,3688,Advice worker,27/05/1973,71a1da150d1ce510193d7622e08e784e,1371816970,41.546067,-78.120238,0
9,21/06/2020 12:16,3.546900e+15,fraud_Kilback Group,food_dining,66.54,Kayla,Obrien,F,7921 Robert Port Suite 343,Prairie Hill,...,31.6591,-96.8094,263,Barrister,30/05/1956,a7915132c7c4240996ba03a47f81e3bd,1371816971,31.782919,-96.366185,0


In [19]:
df_train.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud'],
      dtype='object')

In [20]:
df_train.rename(columns={"trans_date_trans_time":"transaction_time",
                         "amt":"amount(usd)",
                         "trans_num":"transaction_id"},
                inplace=True)
#Convert datetime columns
df_train["transaction_time"] = pd.to_datetime(df_train["transaction_time"], infer_datetime_format=True)
df_train["dob"] = pd.to_datetime(df_train["dob"], infer_datetime_format=True)


C:\Users\User\AppData\Local\Temp\ipykernel_41184\1187565153.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_train["transaction_time"] = pd.to_datetime(df_train["transaction_time"], infer_datetime_format=True)
C:\Users\User\AppData\Local\Temp\ipykernel_41184\1187565153.py:6: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_train["transaction_time"] = pd.to_datetime(df_train["transaction_time"], infer_datetime_format=True)
C:\Users\User\AppData\Local\Temp\ipykernel_41184\1187565153.py:7: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://

In [21]:
from datetime import datetime

# Apply function utcfromtimestamp and drop column unix_time
df_train['time'] = df_train['unix_time'].apply(datetime.utcfromtimestamp)
df_train.drop('unix_time', axis=1)

# Add cloumn hour of day
df_train['hour_of_day'] = df_train.time.dt.hour
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555719 entries, 0 to 555718
Data columns (total 24 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   transaction_time  555719 non-null  datetime64[ns]
 1   cc_num            555719 non-null  float64       
 2   merchant          555719 non-null  object        
 3   category          555719 non-null  object        
 4   amount(usd)       555719 non-null  float64       
 5   first             555719 non-null  object        
 6   last              555719 non-null  object        
 7   gender            555719 non-null  object        
 8   street            555719 non-null  object        
 9   city              555719 non-null  object        
 10  state             555719 non-null  object        
 11  zip               555719 non-null  int64         
 12  lat               555719 non-null  float64       
 13  long              555719 non-null  float64       
 14  city

In [22]:
np.round(df_train.describe(), 2)

,transaction_time,cc_num,amount(usd),zip,lat,long,city_pop,dob,unix_time,merch_lat,merch_long,is_fraud,time,hour_of_day
count,555719,5.557190e+05,555719.00,555719.00,555719.00,555719.00,555719.00,555719,5.557190e+05,555719.00,555719.00,555719.00,555719,555719.00
mean,2020-10-02 01:53:55.673389056,4.178387e+17,69.39,48842.63,38.54,-90.23,88221.89,1973-11-11 16:30:05.937173264,1.380679e+09,38.54,-90.23,0.00,2013-10-02 01:54:25.166780416,12.81
min,2020-06-21 12:14:00,6.041621e+10,1.00,1257.00,20.03,-165.67,23.00,1924-10-30 00:00:00,1.371817e+09,19.03,-166.67,0.00,2013-06-21 12:14:25,0.00
25%,2020-08-09 06:10:00,1.800430e+14,9.63,26292.00,34.67,-96.80,741.00,1962-09-27 00:00:00,1.376029e+09,34.76,-96.91,0.00,2013-08-09 06:10:27.500000,7.00
50%,2020-10-03 00:59:00,3.521420e+15,47.29,48174.00,39.37,-87.48,2408.00,1975-11-30 00:00:00,1.380762e+09,39.38,-87.45,0.00,2013-10-03 00:59:48,14.00
75%,2020-12-01 03:02:30,4.635330e+15,83.01,72011.00,41.89,-80.18,19685.00,1987-04-23 00:00:00,1.385867e+09,41.95,-80.26,0.00,2013-12-01 03:03:04,19.00
max,2020-12-31 23:59:00,4.992350e+18,22768.11,99921.00,65.69,-67.95,2906700.00,2005-01-29 00:00:00,1.388534e+09,66.68,-66.95,1.00,2013-12-31 23:59:34,23.00
std,NaN,1.309837e+18,156.75,26855.28,5.06,13.72,300390.89,NaN,5.201104e+06,5.10,13.73,0.06,NaN,6.81


In [23]:
groups = ['is_fraud', 'job']
df_ = df_train.groupby(by=groups).agg({
    "amount(usd)": 'mean',
    "transaction_id": "count"
}).fillna(0).reset_index()

df_.rename(columns={"amount(usd)": "mean_amount", "transaction_id": "transaction_count"}, inplace=True)

df_ = df_[df_.is_fraud == 1]

# Sort by transaction count in descending order
df_ = df_.sort_values(by='transaction_count', ascending=False)
print(df_)


     is_fraud                          job  mean_amount  transaction_count
614         1               Science writer   471.388667                 30
576         1         Licensed conveyancer   577.833793                 29
630         1            Systems developer   514.688276                 29
537         1         Engineer, biomedical   531.031071                 28
641         1      Therapist, occupational   601.661852                 27
..        ...                          ...          ...                ...
648         1                 Video editor   744.217500                  4
511         1  Clinical research associate   271.806667                  3
514         1    Commercial horticulturist   584.346667                  3
604         1     Public relations officer   785.336667                  3
592         1               Pilot, airline   769.620000                  2

[177 rows x 4 columns]


In [24]:
features = ['transaction_id', 'hour_of_day', 'category', 'amount(usd)', 'merchant', 'job']

#
X = df_train[features].set_index("transaction_id")
y = df_train['is_fraud']

print('X shape:{}\ny shape:{}'.format(X.shape,y.shape))

X shape:(555719, 5)
y shape:(555719,)


In [25]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder(dtype=np.int64)
enc.fit(X.loc[:,['category','merchant','job']])

X.loc[:, ['category','merchant','job']] = enc.transform(X[['category','merchant','job']])

In [26]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

def select_features(X_train, y_train, X_test):
    fs = SelectKBest(score_func=chi2, k='all')
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
print('X_train shape:{}\ny_train shape:{}'.format(X_train.shape,y_train.shape))
print('X_test shape:{}\ny_test shape:{}'.format(X_test.shape,y_test.shape))

X_train shape:(444575, 5)
y_train shape:(444575,)
X_test shape:(111144, 5)
y_test shape:(111144,)


In [27]:

from sklearn.tree import DecisionTreeClassifier

dcstree = DecisionTreeClassifier(random_state=42)
dcstree.fit(X_train, y_train)
y_pred = dcstree.predict(X_test)

# fig = plt.figure(figsize=(8,6))
# cfs_matrix = confusion_matrix(y_test, y_pred)
# sns.heatmap(cfs_matrix, cmap='viridis', annot=True, fmt='d', annot_kws=dict(fontsize=4))


In [28]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    110715
           1       0.64      0.68      0.66       429

    accuracy                           1.00    111144
   macro avg       0.82      0.84      0.83    111144
weighted avg       1.00      1.00      1.00    111144



In [29]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy={1:442859}, random_state=42)

X_train_smote, y_train_smote = smote.fit_resample(X_train.astype('float'), y_train)
print("Before SMOTE:", Counter(y_train))
print("After SMOTE:", Counter(y_train_smote))

Before SMOTE: Counter({0: 442859, 1: 1716})
After SMOTE: Counter({0: 442859, 1: 442859})


In [30]:
class test_model:
    from sklearn.metrics import classification_report
    def __init__(self):
        self.metrics = ['prfs','auc','acc']

    def fit_predict(model, X_train, X_test, y_train, y_test):
        model = model
        model.fit(X_train, y_train) #Fit data to model
        y_pred = model.predict(X_test)
        return y_pred

    def evaluate(y_pred, metrics):
        results = {}
        for metric in metrics:
            if metric == 'prfs':
                prfs = classification_report(y_test, y_pred)
                results['prfs'] = prfs
            elif metric =='auc':
                auc_score = roc_auc_score(y_test, y_pred)
                results['auc'] = auc_score
            elif metric =='acc':
                results['acc'] = accuracy_score(y_test, y_pred)
            else:
                print('Not available metric!')
        return results

from sklearn.ensemble import RandomForestClassifier

# Specify your metric here
metrics = ['prfs']
print("=====================SMOTE=====================")
RDForest_eval = test_model.evaluate(y_pred=test_model.fit_predict(RandomForestClassifier(random_state=42),
                                                                  X_train_smote,
                                                                  X_test,
                                                                  y_train_smote,
                                                                  y_test
                                                                 ),
                                    metrics=metrics
                                    )

print("Random Forest model evaluate:\n", RDForest_eval['prfs'])

=====================SMOTE=====================
Random Forest model evaluate:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    110715
           1       0.64      0.71      0.68       429

    accuracy                           1.00    111144
   macro avg       0.82      0.85      0.84    111144
weighted avg       1.00      1.00      1.00    111144



In [31]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 200, num = 4)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf
              }

print(random_grid)

{'n_estimators': [50, 100, 150, 200], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}


In [32]:
rf_random = RandomForestClassifier(max_features='sqrt',
                                   n_estimators=200,
                                   random_state=41
                                   )

rf_random.fit(X_train, y_train)
y_pred = rf_random.predict(X_test)

# Print reprort
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    110715
           1       0.86      0.66      0.75       429

    accuracy                           1.00    111144
   macro avg       0.93      0.83      0.87    111144
weighted avg       1.00      1.00      1.00    111144



In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# Sample generated transaction


data = {
    "trans_date_trans_time": ["2024-12-07 22:05:15"],
    "cc_num": [6564459919350820],
    "merchant": ["fraud_Goyette, Howell and Collier"],
    "category": ["home"],
    "amt": [800.52],
    "first": ["Douglas"],
    "last": ["Willis"],
    "gender": ["M"],
    "street": ["619 Jeremy Garden Apt. 681"],
    "city": ["Benton"],
    "state": ["WI"],
    "zip": [53803],
    "lat": [42.5545],
    "long": [-90.3508],
    "city_pop": [1306],
    "job": ["Science writer"],
    "dob": ["1958-09-10"],
    "trans_num": ["21313"],
    "unix_time": [1371853942],
    "merch_lat": [42.461127],
    "merch_long": [-91.147148],
}

new_transaction_data = pd.DataFrame(data)
new_transaction_data = new_transaction_data.rename(columns={"trans_date_trans_time":"transaction_time",
                         "cc_num":"credit_card_number",
                         "amt":"amount(usd)",
                         "trans_num":"transaction_id"}
                        )

# Apply function utcfromtimestamp and drop column unix_time
new_transaction_data['time'] = new_transaction_data['unix_time'].apply(datetime.utcfromtimestamp)

# Add cloumn hour of day
new_transaction_data['hour_of_day'] = new_transaction_data.time.dt.hour
features = ['transaction_id', 'hour_of_day', 'category', 'amount(usd)', 'merchant', 'job']
new_transaction_data = new_transaction_data[features].set_index("transaction_id")
enc = OrdinalEncoder(dtype=np.int64)
enc.fit(new_transaction_data.loc[:, ['category','merchant','job']])

new_transaction_data.loc[:, ['category','merchant','job']] = enc.transform(new_transaction_data[['category','merchant','job']])





try:
    fraud_proba = rf_random.predict_proba(new_transaction_data)[:, 1]  # Probability of fraud
    fraud_pred = rf_random.predict(new_transaction_data)              # Fraud prediction (0 or 1)

    # Display results
    new_transaction_data["Fraud_Proba"] = fraud_proba
    new_transaction_data["Fraud_Predict"] = fraud_pred

    print("Prediction Results:")
    print(new_transaction_data[["Fraud_Proba", "Fraud_Predict"]])
except Exception as e:
    print(f"Error during prediction: {e}")


Prediction Results:
                Fraud_Proba  Fraud_Predict
transaction_id                            
21313                  0.59              1


In [36]:
import pickle

# Save the model using pickle
with open("model.pkl", "wb") as file:
    pickle.dump(rf_random, file)